# Pulsating Aurora

This notebook will run the ISR simulator with plasma state parameters that occur during pulsating aurora.  

## Import Stuff

Mainly import matplotlib, scipy and other packages for processing and plotting. Along with functions from the SimISR module.

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
#SimISR packages
from SimISR.utilFunctions import readconfigfile,makeconfigfile
from SimISR.runsim import main as runsim 
from SimISR.IonoContainer import IonoContainer,MakeTestIonoclass
from SimISR.analysisplots import analysisdump

/Users/Bodangles/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Input Parameter Set Up

The following cell will set up the pulsating aurora input plasma parameters. The model of the parameters will be Ne(1+ehn_lev*sin(2*pi*t/PT))

In [ ]:
# Pulsating aurora parameters
PT=30.# the period of pulsation in seconds.
ehn_lev=2. # level of enhancement.
pa_range=[250.,300.]# range of pulsation along altitude in km

# set up the time and space vectors
dt=1.
timevec=sp.arange(120.)*dt
z = (50.+sp.arange(120)*5.)
nz = len(z)
coords = sp.column_stack((sp.zeros((nz,2)),z))
# Create the weighting for the pulsation
logar=sp.logical_and((z>=pa_range[0]),(z<pa_range[1]))
PA_weight=1+.5*(ehn_lev-1)*(1-sp.cos(2.*sp.pi*timevec/PT))
PA_weight_rep = sp.tile(PA_weight[sp.newaxis,:],[logar.sum(),1])

# Make a set of input parameters with a electron density modeled by a Chapman function and an electron and ion temperature
# modeled as shifted and scalled atan functions.
Icont1=MakeTestIonoclass(testv=False,testtemp=True,N_0=1e11,z_0=250.0,H_0=50.0,coords=coords,times=timevec)
# Apply windowing to the electron density 
Icont1.Param_List[logar,:,-1,0]=PA_weight_rep*Icont1.Param_List[logar,:,-1,0]

# Create the starting point data for the fitter.
Icontstart = MakeTestIonoclass(testv=False,testtemp=False,N_0=1e11,z_0=250.0,H_0=50.0,
                               coords=coords,times =sp.array([[0,1e6]]))

## Set up for Simulation

Set up parameters for simulation like pulse length, pulse type, beam positions, 

In [ ]:
# set the number of pulses
npulses = 2000
curloc = Path.cwd()
testpath = curloc.parent.joinpath('Testdata','PulsatingNotebook')
testpath.mkdir(exist_ok=True,parents=True)

defaultpath = curloc.parent.joinpath('Test')
defcon = defaultpath.joinpath('statsbase.ini')
(sensdict,simparams) = readconfigfile(str(defcon))

tint = simparams['IPP']*npulses
ratio1 = tint/simparams['Tint']
simparams['Tint']=ratio1 * simparams['Tint']
simparams['Fitinter'] = ratio1 * simparams['Fitinter']
simparams['TimeLim'] = tint

simparams['startfile']='startfile.h5'
makeconfigfile(str(testpath.joinpath('stats.ini')),simparams['Beamlist'],sensdict['Name'],simparams)

In [ ]:
finalpath = testpath.joinpath('Origparams')
finalpath.mkdir(exist_ok=True,parents=True)


finalfile = os.path.join(finalpath,'0 stats.h5')
Icont1.saveh5(finalfile)
Icontstart.saveh5(os.path.join(testpath,'startfile.h5'))

In [ ]:
functlist = ['spectrums','radardata','fitting']

config = os.path.join(testpath,'stats.ini')

runsim(functlist,testpath,config,True)

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook")
fig1,axmat =plt.subplots(1,3,figsize = (16,7),sharey=True)
axvec = axmat.flatten()
fittedfile = os.path.join(testpath,'Fitted','fitteddata.h5')
fitiono = IonoContainer.readh5(fittedfile)
paramlist = ['Ne','Te','Ti']
indlist =[sp.argwhere(ip==fitiono.Param_Names)[0][0] for ip in paramlist]
n_indlist =[sp.argwhere(('n'+ip)==fitiono.Param_Names)[0][0] for ip in paramlist]

altin =Icont1.Cart_Coords[:,2]
altfit = fitiono.Cart_Coords[:,2]

in_ind=[[1,0],[1,1],[0,1]]
pbounds = [[1e10,1.2e11],[200.,3000.],[200.,2500.],[-100.,100.]]
for i,iax in enumerate(axvec):
    iinind = in_ind[i]
    ifitind = indlist[i]
    n_ifitind = n_indlist[i]
    #plot input
    indata = Icont1.Param_List[:,0,iinind[0],iinind[1]]
    iax.plot(indata,altin)
    #plot fitted data
    fitdata = fitiono.Param_List[:,0,ifitind]
    fit_error = fitiono.Param_List[:,0,n_ifitind]
    ploth=iax.plot(fitdata,altfit)[0]
    iax.set_xlim(pbounds[i])
    iax.errorbar(fitdata,altfit,xerr=fit_error,fmt='-o',color=ploth.get_color())
    iax.set_title(paramlist[i])